# Task 5: Use batch transform to get inferences from a large dataset

## Task 5.1: Environment setup

Install packages and dependencies.

In [ ]:
#install-dependencies
import boto3
import sagemaker
import sagemaker_datawrangler
import time
from sagemaker.session import Session
from sagemaker.transformer import Transformer

role = sagemaker.get_execution_role()
region = boto3.Session().region_name
sess = boto3.Session()
sm = sess.client('sagemaker')
prefix = 'sagemaker/mlasms'
sagemaker_session = sagemaker.Session()
bucket = sagemaker.Session().default_bucket()
s3_client = boto3.client("s3")

Save the model from the training and tuning lab in the default Amazon Simple Storage Service (Amazon S3) bucket. Set up a model using **create_model** and configure **ModelDataUrl** to reference the trained model.

In [ ]:
#set-up-model
# Upload the model and dataset to your Amazon S3 bucket
s3_client.upload_file(Filename="model.tar.gz", Bucket=bucket, Key=f"{prefix}/models/model.tar.gz")

# Set a date to use in the model name
create_date = time.strftime("%Y-%m-%d-%H-%M-%S")
model_name = 'income-model-{}'.format(create_date)

# Retrieve the container image
container = sagemaker.image_uris.retrieve(
    region=boto3.Session().region_name, 
    framework='xgboost', 
    version='1.5-1'
)

# Set up the model
income_model = sm.create_model(
    ModelName = model_name,
    ExecutionRoleArn = role,
    PrimaryContainer = {
        'Image': container,
        'ModelDataUrl': f's3://{bucket}/{prefix}/models/model.tar.gz',
    }
)

Upload the batch records to the default Amazon S3 bucket.

In [ ]:
#upload-dataset
s3_client.upload_file(Filename="batch_data.csv", Bucket=bucket, Key=f"{prefix}/batch_data.csv", ExtraArgs={"ContentType": "text/csv;charset=utf-8"})
batch_path = f"s3://{bucket}/{prefix}/batch_data.csv"

## Task 5.2: Create a batch transform job

Batch transform automatically manages the processing of large datasets within the limits of specified parameters. When a batch transform job starts, SageMaker initializes compute instances and distributes the inference or preprocessing workload between them. Batch transform partitions the Amazon S3 objects in the input by key and maps Amazon S3 objects to instances.

Use batch transform when you need to get inferences from large datasets or when you don't need a persistent endpoint.

To create a batch transform job, you need to set the following options:
- **model_name**: The name of your model.
- **instance_type**: The type of Amazon Elastic Compute Cloud (Amazon EC2) instance to use; for example, "ml.c4.xlarge".
- **instance_count**: The number of EC2 instances to use.
- **assemble_with**: The way in which the output is assembled. Valid values are "Line" or "None".
- **strategy**: The strategy used to decide how to batch records in a single request. Valid values are "MultiRecord" and "SingleRecord".
- **accept**: The file type to accept.
- **output_path**: The Amazon S3 location for saving the transform result. If not specified, results are stored to a default bucket.

In [ ]:
#create-batch-transformer
transformer = Transformer(
    model_name=model_name,
    instance_type="ml.m4.xlarge",
    instance_count=1,
    assemble_with="Line",
    strategy="MultiRecord",
    accept="text/csv",
    output_path="s3://{}/{}/batch-transform/test".format(bucket, prefix)
)

Use the test dataset as your customer records and run the batch transform job. The job can take as long as 10 minutes to run with this set of customer records.

Refer to [Use Batch Transform](https://docs.aws.amazon.com/sagemaker/latest/dg/batch-transform.html) for more information about batch transform jobs.

In [ ]:
#run-batch-transform-job
transformer.transform(batch_path, content_type="text/csv", split_type="Line", join_source="Input")
transformer.wait()

## Task 5.3: View the prediction data in Amazon S3

The batch transform job stores the output in the bucket and folder that you specified when you set up the transformer. You can view the prediction results in Amazon S3, either in the AWS Management Console or in the notebook.

If you want to download and view the output from the console, navigate to Amazon S3, open the bucket starting with **sagemaker-**, and navigate to the object located in **/sagemaker/mlasms/batch-transform/test**. Download the **batch_data.csv.out** object and open it with a notepad editor. The file contains hundreds of predicted values for the customer records that you ran through the batch transform job.

A sample of the output can also be displayed in the notebook.


In [ ]:
!aws s3 cp --recursive $transformer.output_path ./
!head batch_data.csv.out

### Conclusion

Congratulations! You have used Amazon SageMaker to successfully run a batch transform job.

### Cleanup

You have completed this notebook. To move to the next part of the lab, do the following:

- Close this notebook file.
- Return to the lab session and continue with the **Conclusion**.